#  Satellite Tracker Project

### This project involves building a Satellite Tracker using Python and AI to predict and visualize satellite positions based on TLE (Two-Line Element) data from CelesTrak.

-  Project Overview
We will:
- Fetch real-time TLE data from CelesTrak.
- Use Skyfield to calculate the satellite’s position.
- Train an AI model (LSTM) to predict future positions.
- Visualize the satellite’s path on an interactive map.


-  Required Tools & Libraries
-  Programming Language: Python
-  Libraries:

- requests → Fetching TLE data
- skyfield → Satellite orbit calculations
- numpy, pandas → Data processing
- tensorflow/keras → AI model (LSTM)
- matplotlib/plotly → Visualization
- folium → Interactive map

###  Steps to Implement
1- Fetch Satellite Data from CelesTrak
We get the TLE data for a satellite (e.g., ISS - International Space Station).

In [23]:
import requests

# Fetch TLE data for ISS
url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=25544&FORMAT=tle"
response = requests.get(url)
tle_data = response.text.strip().split("\n")

# Extract lines
satellite_name = tle_data[0]
tle_line1 = tle_data[1]
tle_line2 = tle_data[2]

print("Satellite:", satellite_name)
print("TLE Line 1:", tle_line1)
print("TLE Line 2:", tle_line2)


Satellite: ISS (ZARYA)             
TLE Line 1: 1 25544U 98067A   25047.61107277  .00015602  00000+0  27901-3 0  9994
TLE Line 2: 2 25544  51.6365 184.2280 0004243 324.7592  35.3116 15.50178145496463


2- Calculate the Current Orbit using Skyfield
We use Skyfield to compute the satellite’s location at different times.

In [24]:
%pip install skyfield
from skyfield.api import load, EarthSatellite

# Load ephemeris data

ts = load.timescale()
satellite = EarthSatellite(tle_line1, tle_line2, satellite_name, ts)

# Get positions for the next 24 hours
times = [ts.utc(2025, 2, 16, h) for h in range(24)]
positions = [satellite.at(t).subpoint() for t in times]

# Extract latitude, longitude, and altitude
latitudes = [p.latitude.degrees for p in positions]
longitudes = [p.longitude.degrees for p in positions]
altitudes = [p.elevation.km for p in positions]


3- Train an AI Model for Future Predictions
We use an LSTM model to analyze time-series data and predict future positions.

In [25]:
%pip install tensorflow
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Convert data into a training set
data = np.array(list(zip(latitudes, longitudes)))
X_train, y_train = data[:-1], data[1:]

# Build the LSTM model
model = keras.Sequential([
    keras.layers.LSTM(50, return_sequences=True, input_shape=(1, 2)),
    keras.layers.LSTM(50),
    keras.layers.Dense(2)  # Output: (latitude, longitude)
])

model.compile(optimizer='adam', loss='mse')

# Reshape data for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
y_train = y_train.reshape((y_train.shape[0], 2))

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=4)


Note: you may need to restart the kernel to use updated packages.
Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 6180.8281
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6058.4897
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6299.4141
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5653.6460
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5767.2124
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6372.6201
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5714.8384
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7506.4067 
Epoch 9/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5506.9766
Epoch 10/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 6549.0757
Epoch 11/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 4879.1992
Epoch 12/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 5741.3442
Epoch 13/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 5723.6206
Epoch 14/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms

4- Predict Future Positions
We use our trained model to forecast where the satellite will be in the next 6 hours.

In [26]:
# Predict next position
future_prediction = model.predict(X_train[-1].reshape(1, 1, 2))
predicted_lat, predicted_lon = future_prediction[0]

print(f"📍 Predicted Position in 6 Hours:")
print(f"Latitude: {predicted_lat:.2f}, Longitude: {predicted_lon:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 925ms/step
📍 Predicted Position in 6 Hours:
Latitude: 1.40, Longitude: -29.28


5- Visualize the Results on a Map
We use Folium to plot the satellite’s path and the predicted future location.

In [27]:
%pip install folium
import folium
import webbrowser

# Create a map centered at the first recorded position
m = folium.Map(location=[latitudes[0], longitudes[0]], zoom_start=3)

# Plot actual trajectory
for lat, lon in zip(latitudes, longitudes):
    folium.CircleMarker([lat, lon], radius=2, color="blue").add_to(m)

# Plot predicted position
folium.Marker([predicted_lat, predicted_lon], popup="Predicted Position", icon=folium.Icon(color="red")).add_to(m)

# Save and display the map
m.save("satellite_prediction.html")
print(" Map saved! Open satellite_prediction.html to view the results.")
# Open the saved map in the default web browser
webbrowser.open("satellite_prediction.html")

Note: you may need to restart the kernel to use updated packages.
 Map saved! Open satellite_prediction.html to view the results.


True